# Ball Detection Model

In [ ]:
!pip install -U --pre tensorflow=="2.*"

In [ ]:
!pip install pycocotools

In [1]:
import os
import pathlib

In [2]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  28% (1120/3895)
Updating files:  29% (1130/3895)
Updating files:  30% (1169/3895)
Updating files:  31% (1208/3895)
Updating files:  32% (1247/3895)
Updating files:  33% (1286/3895)
Updating files:  34% (1325/3895)
Updating files:  35% (1364/3895)
Updating files:  36% (1403/3895)
Updating files:  37% (1442/3895)
Updating files:  38% (1481/3895)
Updating files:  39% (1520/3895)
Updating files:  40% (1558/3895)
Updating files:  41% (1597/3895)
Updating files:  42% (1636/3895)
Updating files:  43% (1675/3895)
Updating files:  44% (1714/3895)
Updating files:  45% (1753/3895)
Updating files:  45% (1775/3895)
Updating files:  46% (1792/3895)
Updating files:  47% (1831/3895)
Updating files:  48% (1870/3895)
Updating files:  49% (1909/3895)
Updating files:  50% (1948/3895)
Updating files:  51% (1987/3895)
Updating files:  52% (2026/3895)
Updating files:  53% (2065/3895)
Updating files:  54% (2104/3895)
Updating files:  55% (2143/3895)
Updating files:  5

In [3]:
cd models/research

c:\Users\Georg\Downloads\AWS Hack\models\research


In [4]:
!protoc object_detection/protos/*.proto --python_out=.

In [5]:
!pip install object_detection

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --------------- ------------------------ 1.0/2.8 MB 5.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 4.6 MB/s eta 0:00:00
Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 k

In [6]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [7]:
!pip install tf_slim

In [8]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

### Object Detection Model

In [148]:
def load_model(model_name):
    # Updated URL for TF2 models
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_date = '20200711'  # This is the release date for TF2 models
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
        fname=model_name,
        origin=base_url + model_date + '/' + model_file,
        untar=True)
    
    # Search for the saved_model directory
    model_dir_path = pathlib.Path(model_dir)
    
    # Print the directory contents to help debug
    print(f"Downloaded to: {model_dir}")
    print("Directory contents:")
    for path in model_dir_path.glob("**/*"):
        print(path)
    
    # Look for all saved_model directories
    saved_model_paths = list(model_dir_path.glob("**/saved_model"))
    
    if not saved_model_paths:
        raise ValueError(f"Could not find saved_model directory in {model_dir}")
    
    # Use the first saved_model directory found
    saved_model_path = saved_model_paths[0]
    print(f"Found saved_model at: {saved_model_path}")
    
    model = tf.saved_model.load(str(saved_model_path))
    return model.signatures['serving_default']

In [149]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [150]:
# Use this model name
model_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'
detection_model = load_model(model_name)

Downloaded to: C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8
Directory contents:
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\checkpoint
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\pipeline.config
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\saved_model
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\checkpoint\checkpoint
C:\Users\Georg\.keras\datasets\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8\mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-

In [151]:
def run_inference_for_single_image(model, image):
    # Convert image to tensor and expand dimensions
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]
    
    # Run inference
    output_dict = model(input_tensor)
    
    # Process the model outputs
    num_detections = int(output_dict.pop('num_detections'))
    
    # Handle the outputs more robustly to accommodate different model formats
    output_dict = {
        key: value[0, :num_detections].numpy() if len(value.shape) > 1 
        else value[0].numpy()
        for key, value in output_dict.items()
    }
    
    output_dict['num_detections'] = num_detections
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    if 'detection_masks' in output_dict:
        # Reframe detection masks for the image size
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict['detection_masks'], output_dict['detection_boxes'],
            image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

In [152]:
def show_inference(model, image_path,class_id):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  for i,x in enumerate(output_dict['detection_classes']):
    if x==class_id and output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  
  display(Image.fromarray(image_np))

In [153]:
def show_inference_score(model, image_path,class_id):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  for i,x in enumerate(output_dict['detection_classes']):
    if x==class_id and output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)

  if len(scores) == 0:
      return 0
  
  return scores[0]

In [154]:
show_inference_score(detection_model, r"C:\Users\Georg\Downloads\AWS Hack\TestImages\ResizedImages\4\BMG_reitz_rocco_ca90r_2324.png", 37)

0